# Lab: Building a Chat Application with LLM Agentic Tool Mesh

## Introduction

In this lab, we'll explore how to create a robust chat application using Large Language Models (LLMs) with the help of LLM Agentic Tool Mesh's chat services. LLM Agentic Tool Mesh provides all the necessary tools to build a powerful chat system by handling:

- **Prompt Rendering**
- **Model Management**
- **Message Processing**
- **Memory Management**

<div align="center">
  <img src="pictures/chat.png" alt="LLM Agentic Tool Mesh Chat" width="800">
</div>

These services are implemented following the **Factory Design Pattern**, which ensures consistency, scalability, and ease of maintenance. Configuration settings and details of the general services are defined in abstract base classes, while instance-specific settings and behaviors are documented within each concrete implementation.


## Objectives

By the end of this lab, you will:

- Understand how LLM Agentic Tool Mesh simplifies the creation of chat applications using LLMs.
- Learn how to manage models and memories within the chat service.
- Implement prompt rendering and message processing.

## Getting Started

The following bullets must be ran prior to executing notebooks for running this lab:
  1. uv installed and available on PATH with python 3.12 venv

      - Linux/MacOS:
          - `curl -sSL https://get.uv.dev | sh`
          - `source ~/.bashrc`
          - `curl -LsSf https://astral.sh/uv/install.sh | sh`
          - `source $HOME/.local/bin/env`
          - `uv python install 3.12`
          - `uv venv -p 3.12`
          - `source .venv/bin/activate`
          - `uv pip install ipykernel`

      - Windows:
          - TODO
  - Select the venv as the notebook kernel
  <div align="left">
    <img src="pictures/kernel.png" alt="VSCode Juypter UI hint" width="800">
  </div>

**MUST restart Juypter kernel if automated install dependencies cell is ran**
<div align="left">
  <img src="pictures/restart.png" alt="VSCode Juypter UI hint" width="800">
</div>

In [2]:
"""Install platform chat dependencies."""
!cd ../../.. && uv pip install --quiet 'llmesh[chat]'
# Shut down the kernel so user must restart it to apply new pip installations.
# This is a workaround for the fact that Jupyter does not automatically
# pick up new installations in the current kernel.
!echo "Kernel will shut down to apply new pip installations, manual restart required."
import os
os._exit(00)
print("Environment variables loaded from .env")


Kernel will shut down to apply new pip installations, manual restart required.


: 

### Prompt Rendering

As we learned in the previous notebook, Large Language Models (LLMs) rely heavily on **prompts** to generate meaningful responses. A prompt is essentially the input text that the model uses to understand the context and generate an appropriate output. The effectiveness of an LLM is largely determined by how well these prompts are crafted, making **prompt engineering** a critical aspect of working with LLMs.

The **Prompt Rendering** service in LLM Agentic Tool Mesh is designed to simplify and automate the creation and management of prompts, which are essential for interacting with LLMs effectively. This service leverages the concepts we explored earlier to ensure that the prompts fed to the model are well-structured, contextually appropriate, and optimized for desired outputs.

#### Key Features

1. **Dynamic Prompt Creation**: 
   - Allows for generating prompts from string templates and files.
   - Supports placeholders for dynamic content, enabling the insertion of variables (e.g., user input or context) into the prompt structure.
  
2. **Template-Based Rendering**:
   - Uses pre-defined templates to maintain consistent prompt formats, ensuring clarity and coherence.
   - Templates can be customized based on the specific use case, enhancing the effectiveness of the prompt.

3. **File Management**:
   - Enables saving customized prompts back to the file system for reuse.
   - Supports versioning and management of different prompt styles, allowing teams to experiment and find the most effective formats.


In [1]:
from athon.chat import PromptRender

# Example configuration for the Prompt Render
PROMPT_CONFIG = {
    'type': 'JinjaTemplate',
    'environment': 'documents/',
    'templates': {
        'welcome': 'welcome_template.txt',
        'goodbye': 'goodbye_template.txt'
    }
}

# Initialize the Prompt Render with the provided configuration
prompt_render = PromptRender.create(PROMPT_CONFIG)

In [2]:
# Write your name
user_name = "John Doe"

In [3]:
# Render a prompt from a string template
template_string = "Hello, {{ name }}! Welcome to our service."
render_result = prompt_render.render(template_string, name=user_name)

if render_result.status == "success":
    print(f"RENDERED CONTENT:\n{render_result.content}")
else:
    print(f"ERROR:\n{render_result.error_message}")

2025-05-11 15:44:32,759 - ATHON - DEBUG - Prompt generated from string with params {'name': 'John Doe'}
RENDERED CONTENT:
Hello, John Doe! Welcome to our service.


In [4]:
# Load a prompt from a file
load_result = prompt_render.load('welcome', name=user_name)

if load_result.status == "success":
    print(f"LOADED CONTENT:\n{load_result.content}")
else:
    print(f"ERROR:\n{load_result.error_message}")

2025-05-11 15:44:34,949 - ATHON - DEBUG - Prompt generated from documents//welcome_template.txt with params {'name': 'John Doe'}
LOADED CONTENT:
Welcome to our chat service!

Hello John Doe,

I’m Athon, your AI assistant here to help you with any questions or tasks you may have.

Here are a few things I can assist you with:
- Answering questions on a variety of topics.
- Providing summaries and explanations.
- Assisting with translations or information retrieval.

Feel free to ask anything you need!

How can I assist you today?

Best regards,
Athon


### Chat Model Handling

The **Chat Model** service in LLM Agentic Tool Mesh is designed to handle the instantiation and utilization of various Large Language Models (LLMs) based on user-defined configurations. This service is essential for managing different models and ensuring that the appropriate LLM is used for each specific use case.

#### Key Features

1. **Model Instantiation**:
   - Dynamically creates instances of different LLMs (e.g., OpenAI's GPT, Azure GPT) based on configuration parameters.
   - Ensures that the selected model is initialized with the correct settings, such as API keys, temperature, maximum tokens, and other hyperparameters.

2. **Model Utilization**:
   - Provides a standardized interface for interacting with the instantiated LLMs.
   - Manages the execution of prompts and the retrieval of responses from the models.
   - Supports seamless switching between models, allowing for flexibility and experimentation.

In [5]:
from athon.chat import ChatModel
from langchain.schema import HumanMessage, SystemMessage

from src.notebooks.platform_services.lablib.env_util import set_services_env

llm_api_key, llm_model_name, _ = set_services_env()

# Example configuration for the Chat Model
LLM_CONFIG = {
    'type': 'LangChainChatOpenAI',
    'api_key': llm_api_key,
    'model_name': llm_model_name,
    'temperature': 0.7,
}

# Initialize the Chat Model with the provided configuration
chat = ChatModel.create(LLM_CONFIG)

BackgroundProcessManager: atexit handler registered for automated cleanup on kernel exit.
Keeping existing .env file.
2025-05-11 15:44:43,245 - ATHON - DEBUG - Selected Langchain ChatOpenAI


In [6]:
# Define the prompts
prompts = [
    SystemMessage(content="Convert the message to pirate language"),
    HumanMessage(content="Today is a sunny day and the sky is blue")
]

In [7]:
# Invoke the model with the prompts
result = chat.invoke(prompts)

# Handle the response
if result.status == "success":
    print(f"COMPLETION:\n{result.content}")
else:
    print(f"ERROR:\n{result.error_message}")

2025-05-11 15:44:56,706 - ATHON - DEBUG - Prompt generated Arrr, today be a fine sunny day and the sky be a brilliant blue, matey!
COMPLETION:
Arrr, today be a fine sunny day and the sky be a brilliant blue, matey!


### Message Management

The **Message Management** service in LLM Agentic Tool Mesh handles the serialization and deserialization of messages, ensuring smooth communication between different components of the chat system and compatibility across various data formats. This service is critical for maintaining consistency and enabling effective interactions within and outside the LLM Agentic Tool Mesh environment.

#### Key Features

1. **Message Serialization**:
   - Converts message objects into a format suitable for storage or transmission (e.g., JSON, XML).
   - Facilitates the exchange of data between different components of the chat system, such as models, memory services, and prompt renderers.
   - Ensures messages can be sent to external applications or integrated into various web applications, enhancing interoperability.

2. **Message Deserialization**:
   - Transforms serialized data back into message objects that can be processed by the chat system.
   - Maintains compatibility with multiple data formats, allowing the chat system to receive and handle messages from various sources.
   - Supports error handling and validation to ensure that deserialized messages are complete and accurate.

3. **Compatibility Across Platforms**:
   - Enables messages to be exchanged between LLM Agentic Tool Mesh and different web applications, ensuring seamless communication.
   - Provides support for multiple serialization formats, making it easy to integrate the chat system with external APIs and third-party tools.

### Chat Memory

The **Chat Memory** service in LLM Agentic Tool Mesh manages the storage and retrieval of conversation history, which is crucial for maintaining context in chat interactions. This service ensures that the chat application can remember past interactions, enabling it to provide coherent and context-aware responses across multiple turns in a conversation.

#### Key Features

1. **Storage of Conversation History**:
   - Records user inputs and system responses, creating a structured history of the chat session.
   - Supports different memory types (e.g., short-term, long-term) to manage context based on the application's needs.
   - Utilizes efficient data storage techniques to handle large amounts of conversation data without compromising performance.

2. **Retrieval of Context**:
   - Provides quick access to relevant parts of the conversation history, ensuring that the LLM has the necessary context to generate appropriate responses.
   - Supports customizable retrieval strategies, such as retrieving the last few messages or searching for specific keywords in the conversation history.
   - Allows developers to define context windows, determining how much history is retained and referenced in subsequent interactions.